In [12]:
from database.models import HistoricalOHLC
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import tools.tools as tools
import requests
import json
import pandas as pd

### Inicialização do Banco SQLite

In [3]:
historical = HistoricalOHLC()
historical.drop_table()
historical.create_table()

### Extração de Dados da API OHLC (Open, High, Low, Close) e Inserção no Banco

In [4]:
#Configs
today = date.today()
dt_format = "%Y-%m-%d"

BASE_OHLC_URL = 'https://api.coinmarketcap.com/data-api/v3.1/cryptocurrency/historical'

for i in range(0,60):
    
    date_ini = today - relativedelta(months=i+1)
    date_end = today - relativedelta(months=i)
    start_unixtime = tools.to_unixtime_sec(date_ini.strftime(dt_format))
    end_unixtime = tools.to_unixtime_sec(date_end.strftime(dt_format))

    #Extração 
    param = f'?id=1&convertId=2783&timeStart={start_unixtime}&timeEnd={end_unixtime}&interval=1d'
    response = requests.get(BASE_OHLC_URL + param)
    items = response.json()['data']['quotes']

    for item in items:
        #Inserção
        historical.insert((
            item['timeOpen'][0:10],
            item['quote']['open'],
            item['quote']['high'],
            item['quote']['low'],
            item['quote']['close']
        ))


### Extração dos Dados Formatados do Banco de Dados

In [22]:
today = date.today()
dt_format = "%Y-%m-%d"

date_1month = today - relativedelta(months=1)
date_5year = today - relativedelta(years=5)

graph_candle = historical.read_with_filter(f'date between date("{date_1month.strftime(dt_format)}") and date("{today.strftime(dt_format)}")')
graph_all = historical.read_with_filter(f'date between date("{date_5year.strftime(dt_format)}") and date("{today.strftime(dt_format)}")')

df_candle = pd.DataFrame(graph_candle)
df_candle = df_candle.columns[1, 3, 4].rename(columns={1: "Data", 3: "Preco_Max", 4: "Preco_Min" })


     0           1              2              3              4  \
0    1  2023-07-24  143681.477128  143737.516508  136977.841984   
1    2  2023-07-25  137937.745092  139386.420821  137383.939390   
2    3  2023-07-26  138834.050852  140543.417196  138305.635743   
3    4  2023-07-27  139069.492713  140062.814246  137936.749541   
4    5  2023-07-28  138556.216503  139725.327327  138149.710813   
5    6  2023-07-29  138722.026983  139088.226539  138460.476308   
6    7  2023-07-30  138900.152864  139307.412124  137299.669651   
7    8  2023-07-31  138334.178244  139333.073063  137798.195146   
8    9  2023-08-01  138148.026073  142250.623729  137152.516001   
9   10  2023-08-02  142386.827869  143747.464157  139154.559341   
10  11  2023-08-03  140323.722902  143497.824371  139358.849159   
11  12  2023-08-04  143476.692436  143476.692436  140665.806328   
12  13  2023-08-05  141696.000540  141827.956111  141122.930900   
13  14  2023-08-06  141541.577230  142112.930930  141152.35197

### Gráficos